In [ ]:
import plotly
import numpy as np 
import pandas as pd
import plotly.express as px
import matplotlib.pyplot as plt 
import os
import glob
from tqdm import tqdm
import yaml

In [ ]:
df=pd.read_pickle("result.pkl")
df

In [ ]:
#to determine if the MD simulation started from a checkpoint (save the cpt in that case) or outlier
for label in df.sys_label.unique():
    setup=yaml.safe_load(open(f"../md_run/{label}/setting.yml", 'r'))
    if setup['checkpoint']:
        df.loc[(df['sys_label'] == label),['cpt']]= setup['checkpoint'].split('/')[-2]  # sim label from which the cpt came
    elif 'md_run' in setup['pdb_file']:   # started from an outlier
        #print(setup['pdb_file'])
        df.loc[(df['sys_label'] == label),['outlier_MD_label']]=setup['pdb_file'].split('.pd')[0].split('/')[-1].split('_00')[0]
        df.loc[(df['sys_label'] == label),['outlier_MD_frame']]=int(setup['pdb_file'].split('.pd')[0].split('_')[-1])
    df.loc[(df['sys_label'] == label),['wall_clockt']]=int(label.split('_')[-1])  # to add column for starting wall clock times
print (df)

In [ ]:
df.outlier_MD_label.unique()

In [ ]:
print((df['pdb'].value_counts().to_list()))

In [ ]:
# to form md_steps array for the whole df
timeline = {}
md_step=[]
c=0
for gpu_id in df.gpu_id.unique(): 
    sub_df = df[df.gpu_id == gpu_id]
    i=0                      # simulation no. (starting from 0) for the particular GPU id
    for sys_label in sub_df.sys_label.unique(): 
        #current_time = int(sys_label.split('_')[-1]) - 1692329418     # mapping the wall clock starting time of simulation to the beginning of the main job
        #timeline[f"gpu_{gpu_id}"].append(current_time)
        t = list(map(lambda x: x + i*1000, sub_df[sub_df.sys_label == sys_label]['frame'].to_list()))
        c+=1
        #print(len(t),c)
        md_step+=t
        #sub_df.loc[sub_df.sys_label == sys_label]['md_step']=t.copy()
        i+=1

In [ ]:
print(len(md_step),max(md_step))

In [ ]:
len(df)

In [ ]:
df['md_step']=md_step.copy()
df

In [ ]:
# initial weights

df['weight']=1
df

In [ ]:
#changing the weights
print(int(len(df)/6))
mds=0     # will be used to iterate over md steps (multiples of 1000)
while mds < int(len(df)/6):
    for sys_label in df[df.md_step==mds].sys_label.unique():   # iterate over sys_labels at the same md_step
        #print(df[df.sys_label==sys_label])
        if df.loc[(df['sys_label']==sys_label) & (df['frame']==0)]['cpt'].to_list() and mds!=0: # if cpt exists: weight should be the same as that of the continued traj
            gpu_id=str(int(df.loc[(df.sys_label==sys_label) & (df.frame==0)]['gpu_id']))
            #print(gpu_id)
            df.loc[(df.sys_label==sys_label),['weight']]=float(df.loc[(df.gpu_id==gpu_id) & (df.md_step==mds-1)]['weight'])  # weight should be equal to the wt of frame at the previous md_step. 
            #print(df.loc[(df['gpu_id']==gpu_id) & (df['md_step']==mds-1)])
            
            
        # to reweight outlier trajectories
        if sys_label in df.outlier_MD_label.unique():
            sdf=df[(df.outlier_MD_label == sys_label) & (df.frame==0)].sort_values('md_step')   # which simulations had an outlier IC from this sys_label. Need to sort wrt md_step becoz reweighting should be done in the order of outlier detection
            for md_step in sdf.md_step.unique():        
                for outlier_MD_frame in sdf[sdf.md_step==md_step].outlier_MD_frame.unique():
                    fac=len(sdf[(sdf.md_step==md_step) & (sdf.outlier_MD_frame==outlier_MD_frame)])  # how many simulations started from the exact same outlier frame at this md_step 
                    print(fac)
                    df.loc[(df.sys_label==sys_label) & (df.frame >= outlier_MD_frame), ['weight']]/=(fac+1)  # reweight the parent trajectory
                    df.loc[(df.outlier_MD_label==sys_label) & (df.md_step >= md_step) & (df.md_step <= (md_step+999)), ['weight']]= float(df.loc[(df.sys_label==sys_label) & (df.frame == 999)]['weight'])  # reweight the child trajectory(ies)
            #print(sdf)
    #print(mds)
    mds+=1000      # move on to next set of simulations  
#df

In [ ]:
print((df['weight'].value_counts()))

In [ ]:
df.to_pickle("reweighted.pkl")

In [ ]:
df.weight.unique()
